<a href="https://colab.research.google.com/github/trinhvy1411/TRI-TUE-NHAN-TAO/blob/main/BT-TUAN4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
G = [[ 0, 1, 1, 0, 1, 0],
[ 1, 0, 1, 1, 0, 1],
[ 1, 1, 0, 1, 1, 0],
[ 0, 1, 1, 0, 0, 1],
[ 1, 0, 1, 0, 0, 1],
[ 0, 1, 0, 1, 1, 0]]
# Tên các đỉnh của đồ thị.
node = "ABCDEF";
t_={}
for i in range(len(G)):
 t_[node[i]] = i
# Bậc của các đỉnh
degree =[]
for i in range(len(G)):
  degree.append(sum(G[i]))
colorDict = {}
for i in range(len(G)):
 colorDict[node[i]]=["Blue","Red","Yellow","Green"]
# Sắp xếp các đỉnh theo thứ tự bậc
sortedNode=[]
indeks = []
# use selection sort
for i in range(len(degree)):
 _max = 0
 j = 0
 for j in range(len(degree)):
  if j not in indeks:
   if degree[j] > _max:
    _max = degree[j]
    idx = j
 indeks.append(idx)
 sortedNode.append(node[idx])
# Phần xử lý tô màu (sử dụng màu ít nhất có thể)
theSolution={}
for n in sortedNode:
 setTheColor = colorDict[n]
 theSolution[n] = setTheColor[0]
 adjacentNode = G[t_[n]]
 for j in range(len(adjacentNode)):
  if adjacentNode[j]==1 and (setTheColor[0] in colorDict[node[j]]):
   colorDict[node[j]].remove(setTheColor[0])
# In kết quả từng đỉnh và màu đã tô tương ứng
for t,w in sorted(theSolution.items()):
 print("Dinh",t,"=",w)

Dinh A = Yellow
Dinh B = Blue
Dinh C = Red
Dinh D = Yellow
Dinh E = Blue
Dinh F = Red


In [7]:
import random

# --- 1. THIẾT LẬP CÁC HẰNG SỐ VÀ DỮ LIỆU GIẢ LẬP ---

DAYS = ["T2", "T3", "T4", "T5", "T6"]
PERIODS = list(range(1, 6))  # Tiết 1 đến Tiết 5
CLASSES = ["L10", "L11", "L12"]
SUBJECTS = {"Toan": 4, "Tin": 2}  # Tiết/lớp/tuần

# Tổng số tiết cần sắp: (4+2) * 3 = 18 tiết
TOTAL_SLOTS = sum(SUBJECTS.values()) * len(CLASSES)

# Giáo viên: (Tên, Môn)
TEACHERS = [
    ("GVT_1", "Toan"), ("GVT_2", "Toan"), ("GVT_3", "Toan"),
    ("GVTH_1", "Tin"), ("GVTH_2", "Tin")
]

# --- 2. CẤU TRÚC NHIỄM SẮC THỂ (LỊCH TRÌNH) ---

def create_initial_chromosome():
    """Tạo một lịch trình ngẫu nhiên (1 nhiễm sắc thể)"""
    chromosome = []

    # Tạo danh sách các YÊU CẦU DẠY (Slot)
    teaching_requests = []
    for cls in CLASSES:
        for subj, count in SUBJECTS.items():
            for _ in range(count):
                teaching_requests.append((cls, subj))

    # Sắp xếp ngẫu nhiên vị trí và giáo viên cho mỗi yêu cầu
    random.shuffle(teaching_requests)

    # Tạo 25 vị trí (Ngày, Tiết) để làm bộ chứa
    available_time_slots = [(d, p) for d in DAYS for p in PERIODS]
    random.shuffle(available_time_slots) # Dùng ngẫu nhiên 18/25 vị trí

    for i, (cls, subj) in enumerate(teaching_requests):
        # Chọn ngẫu nhiên một GV có thể dạy môn này
        eligible_teachers = [t[0] for t in TEACHERS if t[1] == subj]
        teacher = random.choice(eligible_teachers)

        # Gán vị trí thời gian (Ngày, Tiết)
        day, period = available_time_slots[i]

        # Element: (Lớp, Môn, Ngày, Tiết, Giáo_viên)
        chromosome.append((cls, subj, day, period, teacher))

    return chromosome

# --- 3. HÀM ĐÁNH GIÁ ĐỘ THÍCH NGHI (FITNESS FUNCTION) ---

def calculate_fitness(chromosome):
    """Tính điểm phạt và độ thích nghi (càng cao càng tốt)"""
    penalty = 0

    # Dùng dictionary để kiểm tra các ràng buộc cứng
    teacher_time_slots = {}  # key: (GV, Ngày, Tiết)
    class_time_slots = {}    # key: (Lớp, Ngày, Tiết)

    # Thống kê cho ràng buộc mềm
    teacher_load = {t[0]: 0 for t in TEACHERS}
    class_subject_day_count = {} # key: (Lớp, Môn, Ngày)

    for cls, subj, day, period, teacher in chromosome:
        # --- RÀNG BUỘC CỨNG ---

        # 1. Trùng lặp GV (GV chỉ dạy 1 tiết/thời điểm)
        teacher_key = (teacher, day, period)
        if teacher_key in teacher_time_slots:
            penalty += 100 # Phạt nặng
        teacher_time_slots[teacher_key] = True

        # 2. Trùng lặp Lớp (Lớp chỉ học 1 tiết/thời điểm)
        class_key = (cls, day, period)
        if class_key in class_time_slots:
            penalty += 100 # Phạt nặng
        class_time_slots[class_key] = True

        # --- RÀNG BUỘC MỀM ---

        # 4. Cân bằng tải GV (Sẽ được tính sau)
        teacher_load[teacher] += 1

        # 5. Dàn trải môn học (Không quá 2 tiết cùng môn/ngày/lớp)
        subject_day_key = (cls, subj, day)
        class_subject_day_count[subject_day_key] = class_subject_day_count.get(subject_day_key, 0) + 1
        if class_subject_day_count[subject_day_key] > 2:
            penalty += 5 # Phạt nhẹ

    # Xử lý Ràng buộc mềm 4. Cân bằng tải GV
    avg_load = TOTAL_SLOTS / len(TEACHERS)
    for load in teacher_load.values():
        penalty += abs(load - avg_load) * 2 # Phạt vừa

    # Xử lý Ràng buộc mềm 6. Tiết trống (Logic phức tạp, tạm bỏ qua để giữ code đơn giản)

    # Tính Fitness (Độ thích nghi)
    # 1 / (1 + total_penalty) -> Max fitness = 1 (khi penalty = 0)
    fitness = 1 / (1 + penalty)
    return fitness, penalty

# --- 4. CÁC HÀNH ĐỘNG CỦA GA ---

def selection(population, fitnesses):
    """Lựa chọn: Dùng phương pháp Bánh xe Roulette hoặc Tournament (ở đây dùng đơn giản)"""
    # Tạo danh sách các cá thể có trọng số theo fitness
    weighted_list = [c for c, f in zip(population, fitnesses) if f > 0]
    if len(weighted_list) < 2:
         # Nếu ít quá thì chọn ngẫu nhiên
        return random.choices(population, k=2)
    # Chọn 2 cá thể dựa trên xác suất (Dùng phương pháp roulette)
    return random.choices(population, weights=fitnesses, k=2)

def crossover(parent1, parent2):
    """Lai ghép: Chọn một điểm cắt ngẫu nhiên và trao đổi đuôi"""

    # Lai ghép dựa trên CÁC TIẾT HỌC (chọn 1/2 lịch trình từ P1, 1/2 từ P2)
    # Tuy nhiên, làm vậy dễ bị trùng lặp. Ta dùng Crossover Tiết học-Theo-Lớp.

    child = []

    # Chọn Lớp (ví dụ: L10) sẽ lấy từ parent1, còn lại lấy từ parent2
    cls_to_take_from_p1 = random.choice(CLASSES)

    # Thêm các tiết của lớp đã chọn từ parent1
    for slot in parent1:
        if slot[0] == cls_to_take_from_p1:
            child.append(slot)

    # Thêm các tiết của các lớp còn lại từ parent2
    for slot in parent2:
        if slot[0] != cls_to_take_from_p1:
            child.append(slot)

    # Đảm bảo số lượng tiết là 18, không cần kiểm tra vì cả P1, P2 đều có 18
    return child

def mutation(chromosome, mutation_rate=0.1):
    """Đột biến: Thay đổi ngẫu nhiên Vị trí (Ngày, Tiết) hoặc Giáo viên của một tiết học"""
    new_chromosome = list(chromosome)

    for i in range(len(new_chromosome)):
        if random.random() < mutation_rate:
            cls, subj, day, period, teacher = new_chromosome[i]

            mutation_type = random.choice(["TIME", "TEACHER"])

            if mutation_type == "TIME":
                # Đột biến vị trí (Ngày, Tiết): Đổi chỗ với một tiết học ngẫu nhiên khác
                j = random.randrange(len(new_chromosome))
                # Hoán đổi vị trí (Day, Period) giữa hai tiết i và j
                new_chromosome[i] = (cls, subj, new_chromosome[j][2], new_chromosome[j][3], teacher)
                new_chromosome[j] = (new_chromosome[j][0], new_chromosome[j][1], day, period, new_chromosome[j][4])

            elif mutation_type == "TEACHER":
                # Đột biến Giáo viên: Chọn GV khác cùng môn
                eligible_teachers = [t[0] for t in TEACHERS if t[1] == subj]
                new_teacher = random.choice(eligible_teachers)
                new_chromosome[i] = (cls, subj, day, period, new_teacher)

    return new_chromosome

# --- 5. HÀM CHÍNH GA ---

def genetic_algorithm(
    pop_size=100,
    generations=1000,
    mutation_rate=0.1,
    elitism_count=2
):
    """Khởi chạy Thuật toán Di truyền"""

    # 1. KHỞI TẠO QUẦN THỂ
    population = [create_initial_chromosome() for _ in range(pop_size)]

    best_chromosome = None
    best_fitness = 0.0

    print(f"Bắt đầu GA với {pop_size} cá thể, {generations} thế hệ.")

    for gen in range(generations):
        # 2. ĐÁNH GIÁ ĐỘ THÍCH NGHI
        fitnesses = [calculate_fitness(c)[0] for c in population]

        # Cập nhật cá thể tốt nhất
        current_best_index = fitnesses.index(max(fitnesses))
        current_best_fitness = fitnesses[current_best_index]

        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_chromosome = population[current_best_index]

        if gen % 100 == 0:
            best_penalty = calculate_fitness(best_chromosome)[1]
            print(f"Thế hệ {gen}: Fitness tốt nhất = {best_fitness:.6f} (Penalty: {best_penalty})")

        # Điều kiện dừng
        if best_fitness >= 1.0:
            print(f"Đạt được lịch trình hoàn hảo (Fitness=1.0) ở thế hệ {gen}!")
            break

        # 3. LỰA CHỌN và 6. TẠO QUẦN THỂ MỚI
        new_population = []

        # Elitism: Giữ lại 2 cá thể tốt nhất (ít penalty nhất)
        elite_indices = sorted(range(pop_size), key=lambda i: fitnesses[i], reverse=True)[:elitism_count]
        for i in elite_indices:
            new_population.append(population[i])

        while len(new_population) < pop_size:
            # 3. LỰA CHỌN
            parent1, parent2 = selection(population, fitnesses)

            # 4. LAI GHÉP
            child = crossover(parent1, parent2)

            # 5. ĐỘT BIẾN
            child = mutation(child, mutation_rate)

            new_population.append(child)

        population = new_population

    return best_chromosome, best_fitness

# --- 6. HÀM HIỂN THỊ KẾT QUẢ ---

def print_schedule(chromosome):
    """In lịch trình ra định dạng dễ nhìn"""
    print("\n## Lịch Trình Tối Ưu Nhất ##")

    # Khởi tạo ma trận rỗng (Ngày, Tiết, Lớp)
    schedule_matrix = {cls: {day: {p: "-" for p in PERIODS} for day in DAYS} for cls in CLASSES}
    teacher_loads = {t[0]: 0 for t in TEACHERS}

    for cls, subj, day, period, teacher in chromosome:
        schedule_matrix[cls][day][period] = f"{subj} ({teacher})"
        teacher_loads[teacher] += 1

    for cls in CLASSES:
        print(f"\n### Lịch Lớp {cls} ###")

        # Tạo bảng
        header = ["Tiết"] + DAYS
        rows = []
        for p in PERIODS:
            row = [f"T.{p}"]
            for day in DAYS:
                row.append(schedule_matrix[cls][day][p])
            rows.append(row)

        # Hiển thị bảng
        print("| " + " | ".join(header) + " |")
        print("|" + "---|" * (len(header) + 1))
        for row in rows:
            print("| " + " | ".join(row) + " |")

    print("\n### Tải Giảng Dạy Của Giáo Viên ###")
    for teacher, load in teacher_loads.items():
        print(f"* **{teacher}**: {load} tiết")

# --- CHẠY CHƯƠNG TRÌNH ---
if __name__ == "__main__":
    final_schedule, final_fitness = genetic_algorithm(
        pop_size=500,        # Kích thước quần thể lớn hơn
        generations=2000,    # Số thế hệ lớn hơn
        mutation_rate=0.05   # Tỉ lệ đột biến
    )

    print("\n--- KẾT QUẢ TỐI ƯU HÓA ---")
    final_penalty = calculate_fitness(final_schedule)[1]
    print(f"Fitness cuối cùng: **{final_fitness:.6f}**")
    print(f"Tổng điểm phạt cuối cùng: **{final_penalty}**")

    if final_schedule:
        print_schedule(final_schedule)

Bắt đầu GA với 500 cá thể, 2000 thế hệ.
Thế hệ 0: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 100: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 200: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 300: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 400: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 500: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 600: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 700: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 800: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 900: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 1000: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 1100: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 1200: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 1300: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 1400: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 1500: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 1600: Fitness tốt nhất = 0.172414 (Penalty: 4.8)
Thế hệ 1700: Fitness

Bai 3 BTVN

In [6]:
import itertools
import math

# --- 1. DỮ LIỆU ĐẦU VÀO GIẢ LẬP (Ma trận chi phí) ---

# Tên các thành phố
CITIES = ['A', 'B', 'C', 'D', 'E']
N = len(CITIES)

# Ma trận chi phí (khoảng cách) - Chi phí từ i -> j
# Chi phí tại C[i][j]
# Dùng math.inf cho chi phí không xác định (hoặc từ thành phố đến chính nó)
COST_MATRIX = [
    # A   B   C   D   E
    [math.inf, 10, 15, 20, 8],  # Từ A
    [10, math.inf, 35, 25, 12], # Từ B
    [15, 35, math.inf, 30, 18], # Từ C
    [20, 25, 30, math.inf, 5],  # Từ D
    [8, 12, 18, 5, math.inf]    # Từ E
]

# Tạo map chuyển đổi tên thành phố sang chỉ số (index)
city_to_index = {city: i for i, city in enumerate(CITIES)}
index_to_city = {i: city for i, city in enumerate(CITIES)}

# --- 2. HÀM TÍNH TỔNG CHI PHÍ CỦA MỘT CHU TRÌNH ---

def calculate_path_cost(path):
    """
    Tính tổng chi phí của một chu trình (danh sách các thành phố).
    Chu trình phải quay về điểm xuất phát.

    Ví dụ: path = ['A', 'C', 'B', 'D', 'E'] => A->C->B->D->E->A
    """
    total_cost = 0

    # Chu trình luôn bắt đầu từ thành phố đầu tiên trong danh sách
    start_city = path[0]

    # 1. Tính chi phí đi qua các thành phố
    for i in range(N - 1):
        city1 = path[i]
        city2 = path[i+1]

        idx1 = city_to_index[city1]
        idx2 = city_to_index[city2]

        cost = COST_MATRIX[idx1][idx2]

        # Nếu chi phí là vô hạn, chu trình này không hợp lệ
        if cost == math.inf:
            return math.inf

        total_cost += cost

    # 2. Tính chi phí quay trở lại thành phố ban đầu
    last_city = path[-1]

    idx_last = city_to_index[last_city]
    idx_start = city_to_index[start_city]

    return_cost = COST_MATRIX[idx_last][idx_start]

    if return_cost == math.inf:
        return math.inf

    total_cost += return_cost

    return total_cost

# --- 3. THUẬT TOÁN DUYỆT TOÀN BỘ (BRUTE-FORCE TSP) ---

def solve_tsp_brute_force():
    """
    Duyệt qua tất cả các hoán vị chu trình để tìm chu trình chi phí tối thiểu.
    """
    best_cost = math.inf
    best_path = []

    # Giả sử luôn bắt đầu từ thành phố đầu tiên (A).
    # Ta chỉ cần duyệt các hoán vị của các thành phố còn lại (B, C, D, E).
    start_city = CITIES[0]

    # Danh sách các thành phố cần hoán vị
    other_cities = CITIES[1:]

    # Hàm itertools.permutations tạo ra tất cả hoán vị
    # Ví dụ: permutations(['B', 'C', 'D', 'E'])
    for path_permutation in itertools.permutations(other_cities):

        # Tạo chu trình hoàn chỉnh: [Start] -> [Permutation] -> [Start]
        current_path = [start_city] + list(path_permutation)

        # Tính chi phí của chu trình hiện tại
        current_cost = calculate_path_cost(current_path)

        # Cập nhật kết quả tốt nhất
        if current_cost < best_cost:
            best_cost = current_cost
            best_path = current_path

    # Chu trình cuối cùng: thêm thành phố bắt đầu vào cuối để đóng chu trình
    if best_path:
        final_path_cycle = best_path + [start_city]
    else:
        final_path_cycle = []

    return final_path_cycle, best_cost

# --- CHẠY CHƯƠNG TRÌNH VÀ HIỂN THỊ KẾT QUẢ ---

if __name__ == "__main__":

    print(f"Tổng số thành phố (N): {N}")

    # Số lượng hoán vị cần duyệt: (N-1)! = 4! = 24
    num_permutations = math.factorial(N - 1)
    print(f"Tổng số chu trình cần kiểm tra: {num_permutations}")
    print("-" * 40)

    optimal_path, min_cost = solve_tsp_brute_force()

    print("## KẾT QUẢ TỐI ƯU (BRUTE-FORCE) ")
    if optimal_path:
        print(f"Chu trình chi phí tối thiểu: {' -> '.join(optimal_path)}")
        print(f"Tổng chi phí tối thiểu: {min_cost}")
    else:
        print("Không tìm thấy chu trình hợp lệ.")

Tổng số thành phố (N): 5
Tổng số chu trình cần kiểm tra: 24
----------------------------------------
## KẾT QUẢ TỐI ƯU (BRUTE-FORCE) 
Chu trình chi phí tối thiểu: A -> B -> E -> D -> C -> A
Tổng chi phí tối thiểu: 72


Bai 3
